<a href="https://colab.research.google.com/github/tensorflow/tpu/blob/0e3cfbdfbcf321681c1ac1c387baf7a1a41d8d21/tools/colab/classification_iris_data_with_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2018 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [0]:
# Copyright 2018 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# A simple classification model using Keras with Cloud TPUs


## Overview

This notebook shows how to use Keras to build a simple classification model. The model can train, evaluate, and generate predictions using Cloud TPUs. It uses the iris dataset to predict the species of the flower and also shows how to use your own data instead of using pre-loaded data. This model uses 4 input features ***(SepalLength, SepalWidth, PetalLength, PetalWidth)*** to determine one of these flower species ***(Setosa, Versicolor, Virginica)***.

The model trains for 20 epochs and completes in approximately 2 minutes.

This notebook is hosted on GitHub. To view it in its original repository, after opening the notebook, select **File > View on GitHub**.


**NOTE:** This tutorial is designed to show how to write a simple model using Keras. It should not be used for comparision with training on CPU's because of the very small amount of data being used.

## Learning objectives

In this Colab, you will learn how to:
*   Define a Keras model with 2 hidden layers and 10 nodes in each layer.
*   Create a TPU model from the Keras model.
*   Train, evaluate, and and generate predictions on Cloud TPU.






## Instructions

<h3>  &nbsp;&nbsp;Train on TPU&nbsp;&nbsp; <a href="https://cloud.google.com/tpu/"><img valign="middle" src="https://raw.githubusercontent.com/GoogleCloudPlatform/tensorflow-without-a-phd/master/tensorflow-rl-pong/images/tpu-hexagon.png" width="50"></a></h3>

   1. On the main menu, click Runtime and select **Change runtime type**. Set "TPU" as the hardware accelerator.
   1. Click Runtime again and select **Runtime > Run All**. You can also run the cells manually with Shift-ENTER. 

## Data, model, and training

### Imports

In [0]:
import json
import os
import pandas as pd
import pprint
import tensorflow as tf
import time
import numpy as np
from tensorflow import keras


In [0]:
print(tf.__version__)

### Resolve TPU Address

In [0]:
use_tpu = True #@param {type:"boolean"}

if use_tpu:
    assert 'COLAB_TPU_ADDR' in os.environ, 'Missing TPU; did you request a TPU in Notebook Settings?'

if 'COLAB_TPU_ADDR' in os.environ:
  TF_MASTER = 'grpc://{}'.format(os.environ['COLAB_TPU_ADDR'])
else:
  TF_MASTER=''

with tf.Session(TF_MASTER) as session:
  print ('List of devices:')
  pprint.pprint(session.list_devices())

### FLAGS used as model params

In [0]:
# Model specific parameters

# TPU address
tpu_address = TF_MASTER

# Number of epochs
epochs = 50

# Number of steps_per_epoch
steps_per_epoch = 20

# NOTE: Total number of training steps = Number of epochs * Number of steps_per_epochs

# Total number of evaluation steps. If '0', evaluation after training is skipped
eval_steps = 50

### Download training input data and define prediction input & output

In [0]:
TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"
TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

PREDICTION_INPUT_DATA = {
    'SepalLength': [6.9, 5.1, 5.9, 6.0, 5.5, 6.2, 5.5, 6.3],
    'SepalWidth': [3.1, 3.3, 3.0, 3.4, 2.5, 2.9, 4.2, 2.8],
    'PetalLength': [5.4, 1.7, 4.2, 4.5, 4.0, 4.3, 1.4, 5.1],
    'PetalWidth': [2.1, 0.5, 1.5, 1.6, 1.3, 1.3, 0.2, 1.5],
}

PREDICTION_OUTPUT_DATA = ['Virginica', 'Setosa', 'Versicolor', 'Versicolor', 'Versicolor', 'Versicolor', 'Setosa', 'Virginica']

def maybe_download():
    train_path = tf.keras.utils.get_file(TRAIN_URL.split('/')[-1], TRAIN_URL)
    test_path = tf.keras.utils.get_file(TEST_URL.split('/')[-1], TEST_URL)

    return train_path, test_path

def load_data(y_name='Species'):
    """Returns the iris dataset as (train_x, train_y), (test_x, test_y)."""
    train_path, test_path = maybe_download()

    train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0, dtype={'SepalLength': pd.np.float32,
        'SepalWidth': pd.np.float32, 'PetalLength': pd.np.float32, 'PetalWidth': pd.np.float32, 'Species': pd.np.int32})
    train_x, train_y = train, train.pop(y_name)

    test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0, dtype={'SepalLength': pd.np.float32,
        'SepalWidth': pd.np.float32, 'PetalLength': pd.np.float32, 'PetalWidth': pd.np.float32, 'Species': pd.np.int32})
    test_x, test_y = test, test.pop(y_name)

    return (train_x, train_y), (test_x, test_y)

### Define a Keras model (2 hidden layers with 10 neurons in each)

In [0]:
def get_model():
  return keras.Sequential([
    keras.layers.Dense(10, input_shape=(4,), activation=tf.nn.relu, name = "Dense_1"),
    keras.layers.Dense(10, activation=tf.nn.relu, name = "Dense_2"),
    keras.layers.Dense(3, activation=None, name = "logits"),
    keras.layers.Dense(3, activation=tf.nn.softmax, name = "softmax")
  ])

In [0]:
dnn_model = get_model()
dnn_model.compile(optimizer=tf.train.AdagradOptimizer(learning_rate=0.1), 
              loss='sparse_categorical_crossentropy',
              metrics=['sparse_categorical_crossentropy'])

dnn_model.summary()

### Create a TPU model from a Keras model

In [0]:
tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    dnn_model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(TF_MASTER)))

tpu_model.summary()

### Train the model on TPU

In [0]:
# Fetch the data
(train_x, train_y), (test_x, test_y) = load_data()

# Train the model
tpu_model.fit(
  train_x, train_y,
  steps_per_epoch = steps_per_epoch,
  epochs=epochs,
)

### Evaluation of the model

In [0]:
tpu_model.evaluate(test_x, test_y,
    steps = eval_steps)

### Save the model

In [0]:
tpu_model.save_weights('./DNN_TPU_1024.h5', overwrite=True)

## Prediction

### Prediction data

In [0]:
COLUMNS_NAME=['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
data = pd.DataFrame(PREDICTION_INPUT_DATA, columns=COLUMNS_NAME)
print(data)

### Prediction on TPU

In [0]:
predictions = tpu_model.predict(data)
template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')
for pred_dict, expec in zip(predictions, PREDICTION_OUTPUT_DATA):
  class_index = np.argmax(pred_dict)
  class_probability = np.max(pred_dict)
  print(template.format(SPECIES[class_index], 100*class_probability, expec))

### Prediction on CPU

In [0]:
cpu_model = tpu_model.sync_to_cpu()
cpu_predictions = cpu_model.predict(data)
template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')
for pred_dict, expec in zip(cpu_predictions, PREDICTION_OUTPUT_DATA):
  class_index = np.argmax(pred_dict)
  class_probability = np.max(pred_dict)
  print(template.format(SPECIES[class_index], 100*class_probability, expec))

## What's next

* Learn how to run the same model using [TPUEstimator](https://colab.research.google.com/github/tensorflow/tpu/blob/0e3cfbdfbcf321681c1ac1c387baf7a1a41d8d21/tools/colab/classification_iris_data_with_tpuestimator.ipynb) rather than Keras.
* Learn about [Cloud TPUs](https://cloud.google.com/tpu/docs) that Google designed and optimized specifically to speed up and scale up ML workloads for training and inference and to enable ML engineers and researchers to iterate more quickly.
* Explore the range of [Cloud TPU tutorials and Colabs](https://cloud.google.com/tpu/docs/tutorials) to find other examples that can be used when implementing your ML project.

On Google Cloud Platform, in addition to GPUs and TPUs available on pre-configured [deep learning VMs](https://cloud.google.com/deep-learning-vm/),  you will find [AutoML](https://cloud.google.com/automl/)*(beta)* for training custom models without writing code and [Cloud ML Engine](https://cloud.google.com/ml-engine/docs/) which will allows you to run parallel trainings and hyperparameter tuning of your custom models on powerful distributed hardware.
